Nesta parte da aula, vamos configurar o ambiente de desenvolvimento necessário para criar e treinar Redes Generativas Adversariais (GANs). Vamos usar o Google Colab para isso, e vamos instalar as bibliotecas essenciais, como TensorFlow e Keras.

O Google Colab é uma excelente escolha, pois oferece um ambiente configurado na nuvem com suporte a GPUs, o que é muito útil para treinar modelos de deep learning como GANs.

In [1]:
# Instalando TensorFlow no Google Colab (caso necessário)
# !pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

2025-11-13 18:13:18.559672: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 18:13:18.593788: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-13 18:13:19.396886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Verificando se temos acesso a uma GPU
O que é uma GPU? Uma GPU (Graphics Processing Unit) é um tipo especial de processador que é muito bom em fazer cálculos rápidos, especialmente para tarefas que envolvem muitos dados, como treinar redes neurais.

Por que isso é importante? Quando você está treinando um modelo de inteligência artificial (como o que estamos fazendo aqui), pode demorar muito tempo se você usar apenas o processador comum do seu computador (a CPU). Usar uma GPU pode acelerar esse processo, às vezes tornando-o dezenas de vezes mais rápido.

O que o código faz? Esse pedaço de código verifica se o ambiente na nuvem, como o Google Colab tem uma GPU disponível que possamos usar. Ele simplesmente conta quantas GPUs estão disponíveis e imprime esse número. Se o resultado for zero, significa que estamos usando apenas a CPU, o que pode ser mais lento.

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


2025-11-13 18:13:19.987380: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### Importando o conjunto de dados MNIST

O que é o MNIST? MNIST é um conjunto de dados muito famoso na área de inteligência artificial. Ele contém 70.000 imagens de dígitos escritos à mão (números de 0 a 9). Cada imagem é pequena, com 28 pixels de largura e 28 pixels de altura, e é em preto e branco.

Por que usar o MNIST? Como essas imagens são pequenas e simples, o MNIST é frequentemente usado para testar e treinar modelos de inteligência artificial, especialmente para quem está começando. É como um "caderno de exercícios" para aprender a fazer o computador reconhecer padrões em imagens.

O que o código faz? Aqui, estamos "baixando" essas imagens para o nosso programa. A função tf.keras.datasets.mnist.load_data() carrega as imagens do conjunto de dados MNIST. Nós dividimos esses dados em duas partes: train_images, que são as imagens que usaremos para treinar nosso modelo, e o resto (os rótulos, que identificam quais números são mostrados em cada imagem), que não usaremos por enquanto, por isso os ignoramos com _.

In [4]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

### Normalizando as imagens para o intervalo [-1, 1]

O que é normalização? Normalização é uma técnica usada para ajustar os valores dos dados para que fiquem dentro de um intervalo específico. Isso ajuda o modelo de inteligência artificial a processar os dados de maneira mais eficiente.

Por que normalizar as imagens? As imagens do MNIST são compostas por pixels, e cada pixel tem um valor que vai de 0 a 255 (onde 0 é preto e 255 é branco). Normalizamos esses valores para que fiquem entre -1 e 1, o que facilita o trabalho da rede neural, tornando o aprendizado mais rápido e estável.

O que o código faz?

Reshape: A linha train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') ajusta as imagens para que tenham a forma correta de [28x28 pixels] e uma única camada de cor (já que são imagens em preto e branco). Além disso, converte os valores para o tipo float32, que é um formato de número mais adequado para cálculos precisos.

Normalização: A linha train_images = (train_images - 127.5) / 127.5 normaliza os valores dos pixels. Subtrai-se 127,5 de cada valor de pixel para centralizar os valores em torno de 0 e, em seguida, divide-se por 127,5 para que os valores fiquem no intervalo de -1 a 1.

In [5]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalizando para o intervalo [-1, 1]

### Configurando o buffer e o batch size

O que é buffer? O buffer, neste contexto, é uma espécie de "banco de dados temporário" onde as imagens de treinamento são armazenadas antes de serem enviadas para o modelo de inteligência artificial. Configurar o tamanho do buffer ajuda a garantir que os dados sejam processados de forma eficiente e aleatória.

O que é batch size? Batch size (tamanho do lote) refere-se ao número de imagens que o modelo processa de uma vez antes de atualizar seus "conhecimentos". Em vez de olhar para todas as 60.000 imagens de uma vez (o que seria muito lento e difícil), o modelo processa pequenos lotes de imagens, um lote de cada vez, o que torna o treinamento mais gerenciável e eficiente.

O que o código faz?
BUFFER_SIZE = 60000: Isso define o tamanho do buffer como 60.000, que é o número total de imagens de treinamento. Isso significa que todas as imagens serão armazenadas no buffer, permitindo que o modelo as processe de maneira embaralhada (aleatória) a cada ciclo de treinamento.

BATCH_SIZE = 256: Isso define o tamanho do lote como 256, o que significa que o modelo vai olhar para 256 imagens de cada vez antes de atualizar seus parâmetros. Esse tamanho é um bom equilíbrio entre velocidade e estabilidade durante o treinamento.


In [6]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

### Criando o dataset de treino

O que é um dataset de treino? Um dataset de treino é um conjunto de dados que usamos para "ensinar" um modelo de inteligência artificial. Neste caso, estamos usando as imagens do MNIST para treinar nosso modelo.

O que o código faz?

from_tensor_slices: Essa parte pega todas as imagens de treinamento e as organiza em um formato que o TensorFlow consegue entender.
shuffle: Embaralha as imagens de forma aleatória. Isso é importante porque queremos que o modelo aprenda de maneira equilibrada, sem seguir uma ordem específica das imagens.

batch: Agrupa as imagens em lotes de tamanho BATCH_SIZE (256 imagens por lote, conforme definimos anteriormente). Isso significa que o modelo processará 256 imagens de cada vez durante o treinamento, o que ajuda a acelerar o processo.

Analogia: Imagine que você tem um baralho de cartas. shuffle é como embaralhar o baralho, e batch é como dividir o baralho em pequenos grupos de cartas para facilitar o jogo.

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Construção da Rede Geradora

O que é a Rede Geradora? A rede geradora é uma parte da GAN que cria novas imagens a partir de ruído aleatório. Ela tenta gerar imagens que sejam tão realistas que enganem a outra parte da GAN, a rede discriminadora.

O que o código faz?

model = tf.keras.Sequential(): Estamos criando um modelo sequencial, o que significa que as camadas da rede serão empilhadas uma após a outra.
Dense: A primeira camada Dense pega um vetor de entrada de 100 números (ruído) e o transforma em um vetor maior (7x7x256). Isso é como pegar uma ideia vaga e expandi-la em algo mais concreto.

BatchNormalization: Normaliza os valores dentro da rede, ajudando o modelo a treinar de forma mais eficiente.
LeakyReLU: É uma função de ativação que permite que pequenas quantidades de informação negativa passem, o que ajuda a evitar problemas durante o treinamento.

Reshape: Esta camada reorganiza os dados para que fiquem no formato de uma imagem pequena (7x7 pixels, com 256 camadas).
Conv2DTranspose: Estas camadas aumentam a resolução da imagem, expandindo-a de 7x7 para 14x14, e depois para 28x28, até que a imagem gerada tenha o tamanho final desejado. Pense nisso como uma série de etapas para "refinar" e "ampliar" a imagem.

activation='tanh': A função tanh ajusta os valores de saída para o intervalo [-1, 1], que corresponde ao intervalo em que normalizamos as imagens de treinamento.

Analogia: Imagine que a rede geradora é como um artista que começa com uma ideia vaga (ruído) e, aos poucos, vai detalhando essa ideia até que ela se transforme em uma imagem realista.

In [8]:
# Define uma função que constrói o Gerador (DCGAN) para imagens 28×28×1
def make_generator_model():
    # Modelo sequencial do Keras (empilha camadas em ordem)
    model = tf.keras.Sequential()
    # Projeção do vetor latente z∈ℝ^{100} para 7×7×256
    # (sem bias por estabilidade com BN)
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    # Normalização em lote para estabilizar o treinamento e acelerar a convergência
    model.add(layers.BatchNormalization())
    # Ativação LeakyReLU para evitar "dying ReLU" e manter gradientes fluindo
    model.add(layers.LeakyReLU())

    # Remodela o vetor [7*7*256] para um mapa de ativação [7, 7, 256]
    model.add(layers.Reshape((7, 7, 256)))
    # Upsample #1: mantém 7×7, reduz canais 256→128 com kernel 5×5 e stride 1
    # (deconvolução)
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    # BN para estabilizar estatísticas após a deconvolução
    model.add(layers.BatchNormalization())
    # LeakyReLU para não saturar e manter gradiente
    model.add(layers.LeakyReLU())

    # Upsample #2: dobra a resolução 7×7→14×14, reduz canais 128→64
    # (kernel 5×5, stride 2)
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    # BN novamente para estabilizar
    model.add(layers.BatchNormalization())
    # LeakyReLU na saída intermediária
    model.add(layers.LeakyReLU())

    # Upsample #3 (saída): 14×14→28×28 e mapeia canais 64→1;
    # tanh gera pixels em [-1, 1]
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    # Retorna o gerador pronto para treinar/inferir
    return model

# Cria uma instância do gerador usando a função definida acima
generator = make_generator_model()
# Exibe o resumo da arquitetura (camadas, shapes e nº de parâmetros)
generator.summary()


/home/carloslessa/FCD/POSTECH/modulo4/04-GenerativeIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12544)          │     1,254,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12544)          │        50,176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 7, 7, 128)      │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 14, 14, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 28, 28, 1)      │         1,600 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,330,944 (8.89 MB)

 Trainable params: 2,305,472 (8.79 MB)

 Non-trainable params: 25,472 (99.50 KB)

### Construção da Rede Discriminadora

Função da Rede Discriminadora: A rede discriminadora é uma parte fundamental da GAN (Generative Adversarial Network). Ela funciona como um "crítico" que avalia se uma imagem é real (do conjunto de dados de treino) ou falsa (gerada pela rede geradora). Seu objetivo é distinguir imagens reais das falsas com a maior precisão possível.



Modelo Sequencial: Assim como na rede geradora, aqui também estamos criando um modelo sequencial (Sequential), o que significa que as camadas da rede serão empilhadas uma após a outra.

In [9]:
# Define o Discriminador (DCGAN) que classifica 28×28×1 como real/falso
def make_discriminator_model():
    # Modelo sequencial do Keras (camadas empilhadas)
    model = tf.keras.Sequential()
    # Bloco conv #1: 64 filtros, kernel 5×5, stride 2, mantém spatial com 'same'
    # Input explícito: imagens 28×28×1 (MNIST)
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    # Ativação LeakyReLU (negative_slope≈0.3 por padrão no Keras)
    model.add(layers.LeakyReLU())
    # Dropout para regularizar e reduzir overfitting
    model.add(layers.Dropout(0.3))

    # Bloco conv #2: aprofunda para 128 filtros, reduz mais a resolução
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    # LeakyReLU mantém gradientes em regiões negativas
    model.add(layers.LeakyReLU())
    # Dropout adicional após a ativação
    model.add(layers.Dropout(0.3))

    # Achata mapas de ativação para vetor
    model.add(layers.Flatten())
    # Camada densa final: 1 logit (sem sigmoid) → usar loss com from_logits=True
    model.add(layers.Dense(1))

    # Retorna o discriminador pronto
    return model

# Instancia o discriminador usando a função acima
discriminator = make_discriminator_model()
# Mostra o resumo da arquitetura (shapes e nº de parâmetros)
discriminator.summary()


/home/carloslessa/FCD/POSTECH/modulo4/04-GenerativeIA/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

A Rede Discriminadora é como um crítico rigoroso que analisa imagens e decide se elas parecem reais ou falsas.

Ela usa várias camadas convolucionais para "entender" diferentes aspectos das imagens, e então, no final, decide se a imagem deve ser considerada real ou falsa com base em todos esses aspectos.

O código configura essa rede e a prepara para ser usada junto com a rede geradora no treinamento da GAN, onde as duas redes competirão para melhorar constantemente.

Analogia:
Pense na Rede Discriminadora como um avaliador de diamantes. Ele examina cada diamante (imagem) cuidadosamente, procurando por sinais que indiquem se o diamante é verdadeiro (imagem real) ou falso (imagem gerada). Ele usa várias ferramentas (camadas convolucionais) para fazer essa avaliação e, no final, dá um veredicto (0 ou 1).